## Sending jnl files for checking

In [4]:
import os
import pandas as pd
import numpy as np
import re
from openpyxl import Workbook
try:
        from openpyxl.cell import get_column_letter
except ImportError:
        from openpyxl.utils import get_column_letter
        from openpyxl.utils import column_index_from_string
construction = pd.read_csv("costCentreConstruction_GL.csv")

def finding_GLs(df):
    all_GLs_w_unknown = set(df["GL_account"])
    all_GLs = set([float(i) for i in all_GLs_w_unknown if i != "unknown"])
    
    all_GLs_in_databuild = set([5.11, 5.12, 5.13, 5.14])
    
    # Seeing which GLs are in df
    GL_to_add = sorted(list(all_GLs_in_databuild - all_GLs))
    
    array = []
    for GL in GL_to_add:
        array.append(["OVERHEAD", "OB", "", "JNL",0,"Credit", 0, "31/12/2022","31/12/2022","31/12/2022",GL,False, f"Balancing GL {GL}", ""])
        
    extras = pd.DataFrame(array, columns = ['Job Code', 'Reference', 'Empty', 'Cost Center', 'Empty2', 'Credit',
       'Debit', 'Date', 'Date2', 'Date3', 'GL_account', 'False', 'Comments',
       'Empty3'])
    return pd.concat([df, extras], axis = 'index')

def make_excel(df, file, detailed, test = False):
    df = finding_GLs(df)
    
    wb = Workbook()

    # grab the active worksheet
    ws = wb.active
    
    # Setting up variables
    length_index = len(df)
    
    excel_col = {"Job Code" : "A",
                "Cost Center" : "B",
                "GL_account" : "C",
                "Comments" : "D",
                "Debit" : "E",
                'Credit' : "F",
                'Date' : "G",
                'Reference' : "H",
                'Empty' : "I",
                'Empty2' : "J",
                'False' : "K",
                'Empty3' : "L",
                'Date2' : "M",
                'Date3' : "N",}
    
    sorted_keys = sorted(excel_col, key=excel_col.get)
    df = df[sorted_keys]
    
    # Locations of each column
    job_code = excel_col["Job Code"]
    cost_center = excel_col["Cost Center"]
    GL_account = excel_col["GL_account"]
    comments = excel_col["Comments"]
    debit = excel_col["Debit"]
    credit = excel_col["Credit"]
    date = excel_col["Date"]
    
    reference = excel_col["Reference"]
    empty = excel_col["Empty"]
    empty2 = excel_col["Empty2"]
    false = excel_col["False"]
    empty3 = excel_col["Empty3"]
    date2 = excel_col["Date2"]
    date3 = excel_col["Date3"]

    # Start excel row at 1
    excel_row = 1
    
    need_to_add_formula_list = []
    # Append dataframe column
    ws.append(list(df.columns))
    for num in range(length_index):
        # Adding 1 to excel row for accurate index coordinates
        excel_row += 1
            
        # Obtaining and appending dataframe rows
        row_list = list(df.iloc[num, :])
        ws.append(row_list)
        
        # Taking note of JNL balance pos
        if row_list[1] == "JNL":
            need_to_add_formula_list.append([credit + str(excel_row), row_list[2]])
            
    # Changing value to number
    excel_row = 1
    for num in range(length_index):
        excel_row += 1
        
        value = ws[f"{cost_center}{excel_row}"].value
        
        if value == "JNL":
            continue
            
        if value == "unknown":
            ws[f"{GL_account}{excel_row}"] = f"=VLOOKUP({cost_center}{excel_row}, $S$2:$U$95, 3)"
            continue
            
        ws[f"{cost_center}{excel_row}"] = float(value)
        
        ws[f"{GL_account}{excel_row}"] = f"=VLOOKUP({cost_center}{excel_row}, $S$2:$U$95, 3)"
        
    # Adding formula for JNL balances
    for index in need_to_add_formula_list:
        position = index[0]
        GL_account_num = index[1]
        
        # Add 1 to length_index due to existence of column row
        ws[position] = f'=SUMIFS({debit}2:{debit}{length_index + 1}, {GL_account}2:{GL_account}{length_index + 1}, "={GL_account_num}", {cost_center}2:{cost_center}{length_index + 1}, "<>JNL")'
        
    # Adding a table of cost centers
    index = 1
    ws[f"S{index}"] = "Code"
    ws[f"T{index}"] = "Cost Center"
    ws[f"U{index}"] = "GL Account"
    
    for num in range(len(construction)):
        index += 1
        row_list = list(construction.iloc[num, :])
        
        code = row_list[1]
        cost_center = row_list[2]
        gl_account = row_list[3]
        
        ws[f"S{index}"] = code
        ws[f"T{index}"] = cost_center
        ws[f"U{index}"] = gl_account
        
    ws[f"S{index + 1}"] = "unknown"
    ws[f"T{index + 1}"] = "unknown"
    ws[f"U{index + 1}"] = "unknown"
    
    for column_cells in ws.columns:
        new_column_length = max(len(str(cell.value)) for cell in column_cells)
        new_column_letter = (get_column_letter(column_cells[0].column))
        
        if new_column_length > 15 and new_column_letter != excel_col["Comments"]:
            new_column_length = 15
        if new_column_length > 0:
            ws.column_dimensions[new_column_letter].width = new_column_length*1.2
        
    if test == True:
        wb.save(f"C:/Users/Civic Homes/Desktop/Sample.xlsx")
        return None
    
    # Save the file
    elif detailed == False:
        wb.save(f"G:/My Drive/ESTIMATOR/Journal_Entry/jnl_entries/{file}.xlsx")
    else:
        wb.save(f"G:/My Drive/Estimator/Journal_Entry/jnl_entries_detailed/{file}.xlsx")

# Where jnl code automated files are kept
path = "C:/Users/Civic Homes/Desktop/Jobs/JobsJournalEntries/jnl_files/"

list_jnl_files = os.listdir(path)

for file in list_jnl_files:
    if file == ".ipynb_checkpoints" or "~" in list(file) or file == "Corrected":
        continue
    
    print(file)
    df = pd.read_csv("C:/Users/Civic Homes/Desktop/Jobs/JobsJournalEntries/jnl_files/" + file, sep = "|", header = None)
    df.columns = ["Job Code", "Reference", "Empty", "Cost Center", "Empty2", "Credit", "Debit", "Date", "Date2", "Date3", "GL_account", "False", "Comments", "Empty3"]
    make_excel(df, file, detailed = False)
    
    

1012_Hansel.jnl
101_Barolo.jnl
102_Barolo.jnl
103_Barolo.jnl
104_Alisma.jnl
105_Alisma.jnl
108_Alisma.jnl
1102_Derrimut.jnl
110_Tanninbrae.jnl
1141_Kevin.jnl
122_Dollarbird.jnl
1249_Almon.jnl
1270_Trellis.jnl
127_Litoria.jnl
132_Skyros.jnl
138_Litoria.jnl
1407_Binap.jnl
1447_Brightvale.jnl
1537_Jevons.jnl
156_Litoria.jnl
157_Litoria.jnl
15_Subzero.jnl
1617_Fegent.jnl
176_Fenfield.jnl
17_Lakes.jnl
1802_Azas.jnl
1819_Yellowfinn.jnl
19_Opal.jnl
1_Medway.jnl
215_Alkira.jnl
215_Carnation.jnl
219_Maton.jnl
222_Alister.jnl
2909_Hogans.jnl
32_Grandview.jnl
33_Grandview.jnl
34_Grandview.jnl
352_Walton.jnl
405_Dennison.jnl
427_Pascal.jnl
430_Aaron.jnl
43_Hampshire.jnl
517_Sabro.jnl
518_Bruno.jnl
526_Sabro.jnl
561_Tarneit.jnl
60_Halpin.jnl
628_Harriot.jnl
6361_Edith.jnl
636_Willowleaf.jnl
6407_Window.jnl
6705_Scoparia.jnl
716_Flycatcher.jnl
761_Axle.jnl
884_Scorpio.jnl
929_Romney.jnl


## Sending detailed jnl files

In [5]:
import os
import pandas as pd
import numpy as np
import re
from openpyxl import Workbook
try:
        from openpyxl.cell import get_column_letter
except ImportError:
        from openpyxl.utils import get_column_letter
        from openpyxl.utils import column_index_from_string

def finding_GLs(df):
    all_GLs_w_unknown = set(df["GL_account"])
    all_GLs = set([float(i) for i in all_GLs_w_unknown if i != "unknown"])
    
    all_GLs_in_databuild = set([5.11, 5.12, 5.13, 5.14])
    
    # Seeing which GLs are in df
    GL_to_add = sorted(list(all_GLs_in_databuild - all_GLs))
    
    array = []
    for GL in GL_to_add:
        array.append(["OVERHEAD", "OB", "", "JNL",0,"Credit", 0, "31/12/2022","31/12/2022","31/12/2022",GL,False, f"Balancing GL {GL}", ""])
        
    extras = pd.DataFrame(array, columns = ['Job Code', 'Reference', 'Empty', 'Cost Center', 'Empty2', 'Credit',
       'Debit', 'Date', 'Date2', 'Date3', 'GL_account', 'False', 'Comments',
       'Empty3'])
    return pd.concat([df, extras], axis = 'index')

def make_excel(df, file, detailed, test = False):
    df = finding_GLs(df)
    
    wb = Workbook()

    # grab the active worksheet
    ws = wb.active
    
    # Setting up variables
    length_index = len(df)
    
    excel_col = {"Job Code" : "A",
                "Cost Center" : "B",
                "GL_account" : "C",
                "Comments" : "D",
                "Debit" : "E",
                'Credit' : "F",
                'Date' : "G",
                'Reference' : "H",
                'Empty' : "I",
                'Empty2' : "J",
                'False' : "K",
                'Empty3' : "L",
                'Date2' : "M",
                'Date3' : "N",}
    
    sorted_keys = sorted(excel_col, key=excel_col.get)
    df = df[sorted_keys]
    
    # Locations of each column
    job_code = excel_col["Job Code"]
    cost_center = excel_col["Cost Center"]
    GL_account = excel_col["GL_account"]
    comments = excel_col["Comments"]
    debit = excel_col["Debit"]
    credit = excel_col["Credit"]
    date = excel_col["Date"]
    
    reference = excel_col["Reference"]
    empty = excel_col["Empty"]
    empty2 = excel_col["Empty2"]
    false = excel_col["False"]
    empty3 = excel_col["Empty3"]
    date2 = excel_col["Date2"]
    date3 = excel_col["Date3"]

    # Start excel row at 1
    excel_row = 1
    
    need_to_add_formula_list = []
    # Append dataframe column
    ws.append(list(df.columns))
    for num in range(length_index):
        # Adding 1 to excel row for accurate index coordinates
        excel_row += 1
            
        # Obtaining and appending dataframe rows
        row_list = list(df.iloc[num, :])
        ws.append(row_list)
        
        # Taking note of JNL balance pos
        if row_list[1] == "JNL":
            need_to_add_formula_list.append([credit + str(excel_row), row_list[2]])
            
    # Changing value to number
    excel_row = 1
    for num in range(length_index):
        excel_row += 1
        
        value = ws[f"{cost_center}{excel_row}"].value
        
        if value == "JNL":
            continue
            
        if value == "unknown":
            ws[f"{GL_account}{excel_row}"] = f"=VLOOKUP({cost_center}{excel_row}, $S$2:$U$95, 3)"
            continue
            
        ws[f"{cost_center}{excel_row}"] = float(value)
        
        ws[f"{GL_account}{excel_row}"] = f"=VLOOKUP({cost_center}{excel_row}, $S$2:$U$95, 3)"
        
    # Adding formula for JNL balances
    for index in need_to_add_formula_list:
        position = index[0]
        GL_account_num = index[1]
        
        # Add 1 to length_index due to existence of column row
        ws[position] = f'=SUMIFS({debit}2:{debit}{length_index + 1}, {GL_account}2:{GL_account}{length_index + 1}, "={GL_account_num}", {cost_center}2:{cost_center}{length_index + 1}, "<>JNL")'
        
    # Adding a table of cost centers
    index = 1
    ws[f"S{index}"] = "Code"
    ws[f"T{index}"] = "Cost Center"
    ws[f"U{index}"] = "GL Account"
    
    for num in range(len(construction)):
        index += 1
        row_list = list(construction.iloc[num, :])
        
        code = row_list[1]
        cost_center = row_list[2]
        gl_account = row_list[3]
        
        ws[f"S{index}"] = code
        ws[f"T{index}"] = cost_center
        ws[f"U{index}"] = gl_account
        
    ws[f"S{index + 1}"] = "unknown"
    ws[f"T{index + 1}"] = "unknown"
    ws[f"U{index + 1}"] = "unknown"
    
    for column_cells in ws.columns:
        new_column_length = max(len(str(cell.value)) for cell in column_cells)
        new_column_letter = (get_column_letter(column_cells[0].column))
        
        if new_column_length > 15 and new_column_letter != excel_col["Comments"]:
            new_column_length = 15
        if new_column_length > 0:
            ws.column_dimensions[new_column_letter].width = new_column_length*1.2
        
    if test == True:
        wb.save(f"C:/Users/Civic Homes/Desktop/Sample.xlsx")
        return None
    
    # Save the file
    elif detailed == False:
        wb.save(f"G:\\My Drive\\ESTIMATOR\\Journal_Entry\\jnl_entries\\{file}.xlsx")
    else:
        wb.save(f"G:\\My Drive\\Estimator\\Journal_Entry\\jnl_entries_detailed\\{file}.xlsx")

# Where jnl code automated files are kept
path = "C:/Users/Civic Homes/Desktop/Jobs/JobsJournalEntries/jnl_detailed_files/"

list_jnl_files = os.listdir(path)

for file in list_jnl_files:
    if file == ".ipynb_checkpoints" or "~" in list(file) or file == "Corrected":
        continue
    
    print(file)
    df = pd.read_csv("C:/Users/Civic Homes/Desktop/Jobs/JobsJournalEntries/jnl_detailed_files/" + file, sep = "|", header = None)
    df.columns = ["Job Code", "Reference", "Empty", "Cost Center", "Empty2", "Credit", "Debit", "Date", "Date2", "Date3", "GL_account", "False", "Comments", "Empty3"]
    make_excel(df, file, detailed = True)

1012_Hansel_detailed.jnl
101_Barolo_detailed.jnl
102_Barolo_detailed.jnl
103_Barolo_detailed.jnl
104_Alisma_detailed.jnl
105_Alisma_detailed.jnl
108_Alisma_detailed.jnl
1102_Derrimut_detailed.jnl
110_Tanninbrae_detailed.jnl
1141_Kevin_detailed.jnl
122_Dollarbird_detailed.jnl
1249_Almon_detailed.jnl
1270_Trellis_detailed.jnl
127_Litoria_detailed.jnl
132_Skyros_detailed.jnl
138_Litoria_detailed.jnl
1407_Binap_detailed.jnl
1447_Brightvale_detailed.jnl
1537_Jevons_detailed.jnl
156_Litoria_detailed.jnl
157_Litoria_detailed.jnl
15_Subzero_detailed.jnl
1617_Fegent_detailed.jnl
176_Fenfield_detailed.jnl
17_Lakes_detailed.jnl
1802_Azas_detailed.jnl
1819_Yellowfinn_detailed.jnl
19_Opal_detailed.jnl
1_Medway_detailed.jnl
215_Alkira_detailed.jnl
215_Carnation_detailed.jnl
219_Maton_detailed.jnl
222_Alister_detailed.jnl
2909_Hogans_detailed.jnl
32_Grandview_detailed.jnl
33_Grandview_detailed.jnl
34_Grandview_detailed.jnl
352_Walton_detailed.jnl
405_Dennison_detailed.jnl
427_Pascal_detailed.jnl
430_

## Testing xlsx files

In [42]:
try:
        from openpyxl.cell import get_column_letter
except ImportError:
        from openpyxl.utils import get_column_letter
        from openpyxl.utils import column_index_from_string

def finding_GLs(df):
    all_GLs = set(df["GL_account"])
    print(all_GLs)
    
    all_GLs_in_databuild = set(['5.11', '5.12', '5.13', '5.14'])
    
    # Seeing which GLs are in df
    GL_to_add = sorted(list(all_GLs_in_databuild - all_GLs))
    print(GL_to_add)
    
    array = []
    for GL in GL_to_add:
        array.append(["OVERHEAD", "OB", "", "JNL",0,"Credit", 0, "31/12/2022","31/12/2022","31/12/2022",GL,False, f"Balancing GL {GL}", ""])
        
    extras = pd.DataFrame(array, columns = ['Job Code', 'Reference', 'Empty', 'Cost Center', 'Empty2', 'Credit',
       'Debit', 'Date', 'Date2', 'Date3', 'GL_account', 'False', 'Comments',
       'Empty3'])
    return pd.concat([df, extras], axis = 'index')

def make_excel(df, file, detailed, test = False):
    df = finding_GLs(df)
    
    wb = Workbook()

    # grab the active worksheet
    ws = wb.active
    
    # Setting up variables
    length_index = len(df)
    
    excel_col = {"Job Code" : "A",
                "Cost Center" : "B",
                "GL_account" : "C",
                "Comments" : "D",
                "Debit" : "E",
                'Credit' : "F",
                'Date' : "G",
                'Reference' : "H",
                'Empty' : "I",
                'Empty2' : "J",
                'False' : "K",
                'Empty3' : "L",
                'Date2' : "M",
                'Date3' : "N",}
    
    sorted_keys = sorted(excel_col, key=excel_col.get)
    df = df[sorted_keys]
    
    # Locations of each column
    job_code = excel_col["Job Code"]
    cost_center = excel_col["Cost Center"]
    GL_account = excel_col["GL_account"]
    comments = excel_col["Comments"]
    debit = excel_col["Debit"]
    credit = excel_col["Credit"]
    date = excel_col["Date"]
    
    reference = excel_col["Reference"]
    empty = excel_col["Empty"]
    empty2 = excel_col["Empty2"]
    false = excel_col["False"]
    empty3 = excel_col["Empty3"]
    date2 = excel_col["Date2"]
    date3 = excel_col["Date3"]

    # Start excel row at 1
    excel_row = 1
    
    need_to_add_formula_list = []
    # Append dataframe column
    ws.append(list(df.columns))
    for num in range(length_index):
        # Adding 1 to excel row for accurate index coordinates
        excel_row += 1
            
        # Obtaining and appending dataframe rows
        row_list = list(df.iloc[num, :])
        ws.append(row_list)
        
        # Taking note of JNL balance pos
        if row_list[1] == "JNL":
            need_to_add_formula_list.append([credit + str(excel_row), row_list[2]])
            
    # Changing value to number
    excel_row = 1
    for num in range(length_index):
        excel_row += 1
        
        value = ws[f"{cost_center}{excel_row}"].value
        
        if value == "JNL":
            continue
            
        if value == "unknown":
            ws[f"{GL_account}{excel_row}"] = f"=VLOOKUP({cost_center}{excel_row}, $S$2:$U$95, 3)"
            continue
            
        ws[f"{cost_center}{excel_row}"] = float(value)
        
        ws[f"{GL_account}{excel_row}"] = f"=VLOOKUP({cost_center}{excel_row}, $S$2:$U$95, 3)"
        
    # Adding formula for JNL balances
    for index in need_to_add_formula_list:
        position = index[0]
        GL_account_num = index[1]
        
        # Add 1 to length_index due to existence of column row
        ws[position] = f'=SUMIFS({debit}2:{debit}{length_index + 1}, {GL_account}2:{GL_account}{length_index + 1}, "={GL_account_num}", {cost_center}2:{cost_center}{length_index + 1}, "<>JNL")'
        
    # Adding a table of cost centers
    index = 1
    ws[f"S{index}"] = "Code"
    ws[f"T{index}"] = "Cost Center"
    ws[f"U{index}"] = "GL Account"
    
    for num in range(len(construction)):
        index += 1
        row_list = list(construction.iloc[num, :])
        
        code = row_list[1]
        cost_center = row_list[2]
        gl_account = row_list[3]
        
        ws[f"S{index}"] = code
        ws[f"T{index}"] = cost_center
        ws[f"U{index}"] = gl_account
        
    ws[f"S{index + 1}"] = "unknown"
    ws[f"T{index + 1}"] = "unknown"
    ws[f"U{index + 1}"] = "unknown"
    
    for column_cells in ws.columns:
        new_column_length = max(len(str(cell.value)) for cell in column_cells)
        new_column_letter = (get_column_letter(column_cells[0].column))
        
        if new_column_length > 15 and new_column_letter != excel_col["Comments"]:
            new_column_length = 15
        if new_column_length > 0:
            ws.column_dimensions[new_column_letter].width = new_column_length*1.2
        
    if test == True:
        wb.save(f"C:/Users/Civic Homes/Desktop/Sample.xlsx")
        return None
    
    # Save the file
    elif detailed == False:
        wb.save(f"G:\\My Drive\\ESTIMATOR\\Journal_Entry\\jnl_entries\\{file}.xlsx")
    else:
        wb.save(f"G:\\My Drive\\Estimator\\Journal_Entry\\jnl_entries_detailed\\{file}.xlsx")


make_excel(df, file, detailed = True, test = True)

{'5.12', '5.11', 'unknown', '5.13'}
['5.14']


PermissionError: [Errno 13] Permission denied: 'C:/Users/Civic Homes/Desktop/Sample.xlsx'

In [18]:
def finding_GLs(df):
    all_GLs = set(df["GL_account"])
    
    all_GLs_in_databuild = set(['5.11', '5.12', '5.13', '5.14'])
    
    # Seeing which GLs are in df
    GL_to_add = sorted(list(all_GLs_in_databuild - all_GLs))
    
    array = []
    for GL in GL_to_add:
        array.append(["OVERHEAD", "OB", "", "JNL",0,"Credit", 0, "31/12/2022","31/12/2022","31/12/2022",GL,False, f"Balancing GL {GL}", ""])
        
    extras = pd.DataFrame(array, columns = ['Job Code', 'Reference', 'Empty', 'Cost Center', 'Empty2', 'Credit',
       'Debit', 'Date', 'Date2', 'Date3', 'GL_account', 'False', 'Comments',
       'Empty3'])
    return pd.concat([df, extras], axis = 'index')
        
finding_GLs(df)

,Job Code,Reference,Empty,Cost Center,Empty2,Credit,Debit,Date,Date2,Date3,GL_account,False,Comments,Empty3
0,U04957,OB,NaN,240,0,0.0,14407.26,31/12/2022,31/12/2022,31/12/2022,5.11,False,20MPA 20MM - 83.7CBM - U04957/240 ; Concrete S...,NaN
1,U04957,OB,NaN,440,0,0.0,13712.00,31/12/2022,31/12/2022,31/12/2022,5.11,False,Bricklayer Labour-PO:U04957/440 ; Bricklayer L...,NaN
2,U04957,OB,NaN,320,0,0.0,13468.18,31/12/2022,31/12/2022,31/12/2022,5.11,False,Supply Pre fab roof trusses PO:U04957/320 ; Ro...,NaN
3,U04957,OB,NaN,230,0,0.0,10095.33,31/12/2022,31/12/2022,31/12/2022,5.11,False,Materials for Slab PO: U04957/230 ; Reinforcem...,NaN
4,U04957,OB,NaN,390,0,0.0,5644.80,31/12/2022,31/12/2022,31/12/2022,5.11,False,Completion of house frames - PO:U04957/390 ; C...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,OVERHEAD,OB,NaN,JNL,0,57200.24,0.00,31/12/2022,31/12/2022,31/12/2022,5.12,False,Balancing GL 5.12,NaN
66,U04957,OB,NaN,unknown,0,0.0,37.14,31/12/2022,31/12/2022,31/12/2022,unknown,False,BUNNINGS BILL - REIMBURSEMENT ; Misc. Payments...,NaN
67,U04957,OB,NaN,unknown,0,0.0,30.55,31/12/2022,31/12/2022,31/12/2022,unknown,False,BUNNINGS BILL - REIMBURSEMENT ; Misc. Payments...,NaN
0,OVERHEAD,OB,,JNL,0,Credit,0.00,31/12/2022,31/12/2022,31/12/2022,5.13,False,Balancing GL 5.13,
